In [3]:
import os
from dotenv import load_dotenv

from langchain.agents import AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate, MessagesPlaceholder
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.chat_models import init_chat_model
from langchain_core.messages import SystemMessage, AIMessage, HumanMessage
from langchain.agents import create_tool_calling_agent
from langchain import hub
from langchain_unstructured import UnstructuredLoader
import faiss
from langchain_community.vectorstores import FAISS
from langchain_core.tools import tool
from tqdm.autonotebook import tqdm as notebook_tqdm
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.document_loaders import DirectoryLoader
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint, HuggingFacePipeline

from langchain_ollama import OllamaEmbeddings, ChatOllama
from langchain_huggingface import HuggingFaceEmbeddings
load_dotenv()

/tmp/ipykernel_16339/3676789303.py:15: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm as notebook_tqdm


True

In [4]:
# import transformers

load_dotenv()

llm = ChatOllama(model='qwen2:latest', temperature=0)
embed= OllamaEmbeddings(model='mxbai-embed-large:latest')

In [5]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_unstructured import UnstructuredLoader

page_urls = ["https://hermextravels.com/","https://hermextravels.com/features.html",
            "https://hermextravels.com/contact.html", "https://hermextravels.com/investors.html"]

loader = WebBaseLoader(web_paths=page_urls)
docs = []
async for doc in loader.alazy_load():
    docs.append(doc)

Fetching pages: 100%|##########| 4/4 [00:12<00:00,  3.03s/it]


In [6]:
# Load, chunk and index the contents of the blog.
# docs = DirectoryLoader('source/', use_multithreading=True).load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

In [ ]:
# Create a FAISS index from the documents
vector_store = FAISS.from_documents(
    documents=splits,
    embedding=embed,
)

In [13]:
prompt = hub.pull('hwchase17/openai-functions-agent')

@tool(response_format="content")
def retrieve(query:str):
    """
    Retrieve information related to a query

    Args:
        query (str): user question or comment
    """
    retrieved_docs = vector_store.similarity_search(query, k=1)
    serialized = "".join(
        (f"Source: {doc.metadata}\n" f"Content: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized # , retrieved_docs


tools = [retrieve]
agent = create_tool_calling_agent(llm, tools, prompt)


agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [14]:
response = agent_executor.invoke({'input':'What is Hermex all about?'})
print(response['output'])



> Entering new AgentExecutor chain...

Invoking: `retrieve` with `{'query': 'What is Hermex all about?'}`


Source: {'source': 'https://hermextravels.com/investors.html', 'title': 'For Investors — Hermex Travels', 'description': 'Investment opportunities with Hermex Travels', 'language': 'No language found.'}
Content: Opportunities for Investors
Hermex Travels is at the forefront of travel innovation, integrating AI, AR, and blockchain technology to enhance travel experiences. As we continue to develop, we invite forward-thinking investors to be part of this exciting journey.
Interested in helping us bring the future of travel to life?
				Contact Us for more details on investment opportunities.


















Copyright 2024 Hermex Services Home. All Rights Reserved.Hermex Travels is a company that integrates AI, AR, and blockchain technology to enhance travel experiences. They are inviting forward-thinking investors to be part of their journey towards developing the future of tra

### Hera: AI Assistant

In [ ]:
# hermex_assistant_chatbot.py
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOllama(model='qwen2:latest', temperature=0)
embed = OllamaEmbeddings(model='mxbai-embed-large:latest')

page_urls = [
            "https://hermextravels.com/",
            "https://hermextravels.com/features.html",
            "https://hermextravels.com/contact.html",
            "https://hermextravels.com/investors.html"
        ]

def setup_vectorstore():
    loader = WebBaseLoader(web_paths=page_urls)
    docs = loader.load()
    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
    chunks = splitter.split_documents(docs)
    vectorstore = FAISS.from_documents(chunks, embed)
    return vectorstore

retriever = setup_vectorstore()

class HermexAssistant:
    def __init__(self, streaming:bool=True):
        load_dotenv()
        self.llm = llm
        self.embedding_model = embed
        self.prompt_template : ChatPromptTemplate = ChatPromptTemplate.from_template(
            """
            You are Hera, a professional and efficient human travel assistant for Hermex Travels.
            Your purpose is to provide accurate, concise, and helpful answers strictly based on the provided context.
            Avoid speculation and only provide facts relevant to Hermex services and the travel/tourism industry.

            Use the following context to answer the user's question. If the answer is not in the context, reply with:
            "I'm sorry, I do not have information on that at the moment."

            Context:
            {context}

            Question: {question}

            Answer as a knowledgeable, polite, and articulate human travel assistant.You have access to the following tools:
            {tools}
            Use the following format:

            Question: the input question you must answer
            Thought: you should always think about what to do
            Action: the action to take, should be one of [{tool_names}]. Always look first in Retriever
            Action Input: the input to the action
            Observation: the result of the action
            ... (this Thought/Action/Action Input/Observation can repeat 2 times)
            Thought: I now know the final answer
            Final Answer: the final answer to the original input question

            Begin!

            Question: {input}
            Thought:{agent_scratchpad}
            """
        )
        self.streaming: bool = streaming
        self.user_sessions: dict = {}
        self.conversational_memory = ConversationBufferWindowMemory(
                        memory_key='chat_history',
                        k=5,
                        return_messages=True)

    
    @tool(response_format="content_and_artifact")
    def retriever(query:str):
        """
        Retrieve information related to a query

        Args:
            query (str): user question or comment
        """
        retrieved_docs = retriever.similarity_search(query, k=2)
        serialized = "".join(
            (f"Source: {doc.metadata}\n" f"Content: {doc.page_content}")
            for doc in retrieved_docs
        )
        return serialized, retrieved_docs


    def answer_question(self, user_id:str, query:str)-> str:
            return self._standard_answer(user_id, query)

    def _standard_answer(self, user_id:str, query:str)-> str:
        if user_id not in self.user_sessions:
            self.user_sessions[user_id] = []

        tools = [retriever]
        agent = create_tool_calling_agent(llm, tools, self.prompt_template)

        agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
        response = agent_executor.invoke({'input':query})
        self.user_sessions[user_id].append((query, response['output']))
        return response['output']

    def get_chat_history(self, user_id:str)-> list:
        return self.user_sessions.get(user_id, [])
    
    def clear_chat_history(self, user_id:str):
        if user_id in self.user_sessions:
            del self.user_sessions[user_id]
            return True
        return False
    


In [ ]:
Hera = HermexAssistant(streaming=False)

In [ ]:
import asyncio, nest_asyncio
nest_asyncio.apply()

Hera.answer_question(user_id="user1", query="What is Hermex all about?")

In [ ]:
Hera.answer_question("user1", "how are you?")

In [ ]:
Hera.answer_question("user1", "who are you?")

In [ ]:
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_ENDPOINT = "https://hermex-ai-service.services.ai.azure.com/models/chat/completions?api-version=2024-05-01-preview"  # Use your target URL


In [ ]:
import os
import logging
from dotenv import load_dotenv
from langchain_community.document_loaders import WebBaseLoader, TextLoader
from langchain_community.vectorstores import FAISS
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.memory import ConversationBufferWindowMemory, ChatMessageHistory
from langchain_core.prompts import (ChatPromptTemplate, MessagesPlaceholder, 
                                    HumanMessagePromptTemplate, 
                                    SystemMessagePromptTemplate)
from langchain.agents import tool, create_tool_calling_agent, AgentExecutor
from langchain.chains import RetrievalQA

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Load environment variables
load_dotenv()

# Models
llm = ChatOllama(model='qwen2:latest', temperature=0)
embed = OllamaEmbeddings(model='mxbai-embed-large:latest')

# FAISS path
FAISS_PATH = "hermex_faiss_index"

# URLs to load
page_urls = [
    "https://hermextravels.com/",
    "https://hermextravels.com/features.html",
    "https://hermextravels.com/contact.html",
    "https://hermextravels.com/investors.html",
    "https://hermextravels.com/investors.html#investment-tiers"
]

def setup_vectorstore():
    try:
        if os.path.exists(FAISS_PATH):
            logging.info("🔁 Loading FAISS index from disk...")
            return FAISS.load_local(FAISS_PATH, embed, 
                                    allow_dangerous_deserialization=True)
        else:
            logging.info("🔨 Creating FAISS index from webpages...")

            web_loader = WebBaseLoader(web_paths=page_urls)
            web_docs = web_loader.load()

            text_loader = TextLoader("hermex.txt", encoding="utf-8")
            text_docs = text_loader.load()

            all_docs = web_docs + text_docs
            splitter = RecursiveCharacterTextSplitter(chunk_size=1000, 
                                                      chunk_overlap=200, 
                                                      multithreaded=True)
            chunks = splitter.split_documents(all_docs)

            vectorstore = FAISS.from_documents(chunks, embed)
            vectorstore.save_local(FAISS_PATH)
            logging.info("✅ FAISS index created and saved.")
            return vectorstore
    except Exception as e:
        logging.error(f"Error setting up vectorstore: {e}")
        raise RuntimeError("Vectorstore setup failed.") from e


class HermexAssistant:
    def __init__(self):
        try:
            self.llm = llm
            self.embedding_model = embed
            self.user_sessions: dict[str, ChatMessageHistory] = {}
            self.vectorstore = setup_vectorstore()

            self.instructions = """
                You are Hera, a professional and efficient travel assistant for 
                Hermex Travels. Your purpose is to provide accurate, concise, and 
                helpful answers strictly based on the provided context.
                Avoid speculation and only provide facts relevant to Hermex services
                and the travel/tourism industry.

                Use the following context to answer the user's question. 
                If the answer is not in the context, reply with:
                "I'm sorry, I can not provide such information at the moment."
            """

            prompt = ChatPromptTemplate(
                messages=[
                    SystemMessagePromptTemplate.from_template(self.instructions),
                    MessagesPlaceholder("chat_history"),
                    HumanMessagePromptTemplate.from_template("{input}"),
                    MessagesPlaceholder("agent_scratchpad"),
                ]
            )

            # Define tools
            @tool(response_format="content_and_artifact")
            def retrieve():
                """Retrieve information related to a query to 
                provide an informative response."""
                try:
                    return self.vectorstore.as_retriever(search_kwargs={"k": 2}), \
                            self.conversational_memory.chat_memory
                except Exception as e:
                    logging.error(f"Error during retrieval: {e}")
                    return "Retrieval failed."

            @tool(response_format="content")
            def ip_address():
                """Retrieve IP-address information of the user"""
                try:
                    from requests import get
                    return get('https://api.ipify.org').text
                except Exception as e:
                    logging.error(f"Error fetching IP address: {e}")
                    return "Could not retrieve IP address."

            @tool(response_format="content")
            def date_time():
                """Retrieve current day, date, and time in a readable format"""
                try:
                    from datetime import datetime
                    now = datetime.now()
                    return now.strftime("%A, %Y-%m-%d %H:%M:%S")
                except Exception as e:
                    logging.error(f"Error getting date and time: {e}")
                    return "Could not retrieve date/time."

            self.conversational_memory = ConversationBufferWindowMemory(
                memory_key='chat_history',
                k=5,
                return_messages=True
            )

            self.agent_executor = AgentExecutor(
                agent=create_tool_calling_agent(self.llm, [retrieve, 
                                                           date_time, 
                                                           ip_address], prompt),
                tools=[retrieve, date_time, ip_address],
                memory=self.conversational_memory,
                # verbose=True,
            )
            logging.info("🤖 HermexAssistant initialized successfully.")

        except Exception as e:
            logging.critical(f"Failed to initialize HermexAssistant: {e}")
            raise

    def chat(self, user_id: str, query: str) -> str:
        """Handles chat with a specific user session."""
        try:
            if user_id not in self.user_sessions:
                self.user_sessions[user_id] = ChatMessageHistory()

            self.conversational_memory.chat_memory = self.user_sessions[user_id]
            self.user_sessions[user_id].add_user_message(query)

            response = self.agent_executor.invoke({'input': query})
            output = response.get('output', 
                                  "I'm sorry, I couldn't generate a response.")
            self.user_sessions[user_id].add_ai_message(output)
            return output

        except Exception as e:
            logging.error(f"Chat error for user {user_id}: {e}")
            return "Something went wrong. Please try again later."

    def load_history(self, user_id: str) -> list:
        """Returns user's conversation history or initializes it if missing."""
        try:
            if user_id not in self.user_sessions:
                self.user_sessions[user_id] = ChatMessageHistory()
            return self.user_sessions[user_id].messages
        except Exception as e:
            logging.error(f"Error loading history for user {user_id}: {e}")
            return []

    def clear_chat_history(self, user_id: str):
        try:
            if user_id in self.user_sessions:
                del self.user_sessions[user_id]
                logging.info(f"Cleared chat history for user {user_id}")
                return True
            return False
        except Exception as e:
            logging.error(f"Error clearing chat history for user {user_id}: {e}")
            return False


In [80]:
her = HermexAssistant(streaming=False)
her.chat(user_id="user1", query="What is Hermex all about?")

🔁 Loading FAISS index from disk...


"Hermex Travels is a professional travel assistance service dedicated to providing accurate, concise, and helpful answers based on the provided context. Our focus is on offering information strictly related to Hermex services and the travel/tourism industry without speculation or irrelevant details.\n\nTo provide you with more specific information about Hermex Travels, I would need additional data or context about the aspects of Hermex you are interested in learning about. Please specify if you want to know about their services, destinations they offer, booking process, customer support, or any other relevant topic related to travel assistance and tourism.\n\nIf you're looking for general information on how we operate as a professional service provider within the travel industry, I can confirm that our primary goal is to ensure smooth and efficient travel experiences for our clients by providing them with accurate information and assistance tailored to their needs."

In [81]:
her.chat(user_id="user1", query="Who are you?")

'I am Hera, a professional and efficient travel assistant designed to provide accurate, concise, and helpful answers based on the provided context related to Hermex Travels services and the travel/tourism industry. My purpose is to assist with inquiries about destinations, booking processes, customer support, and other relevant information without speculation or irrelevant details.'

In [82]:
her.chat(user_id="user1", query="what is my ip?")

'Your IP address is 102.89.47.73.'

In [83]:
her.chat(user_id="user1", query="what was my last question?")

'Your last question was: "What is my ip?"'

In [84]:
her.chat(user_id="user1", query="what is today's date?")

"Today's date is 2023-11-28.\n\nToday's date is November 28, 2023."

In [93]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import WebBaseLoader, TextLoader
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_huggingface import HuggingFacePipeline
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.memory import ConversationBufferWindowMemory, ChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate, SystemMessagePromptTemplate
from langchain.agents import tool, create_tool_calling_agent, AgentExecutor
from langchain import hub

# Load environment variables
load_dotenv()

# Set up LLM using Hugging Face Hub
llm = HuggingFacePipeline.from_model_id(
    model_id="google/flan-t5-xxl", 
    task = 'text generation'    # model_kwargs={"temperature": 0.2, "max_length": 512}
)

# Set up embeddings
embed = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

FAISS_PATH = "hermex_faiss_index"

page_urls = [
    "https://hermextravels.com/",
    "https://hermextravels.com/features.html",
    "https://hermextravels.com/contact.html",
    "https://hermextravels.com/investors.html",
    "https://hermextravels.com/investors.html#investment-tiers"
]

def setup_vectorstore():
    if os.path.exists(FAISS_PATH):
        print("🔁 Loading FAISS index from disk...")
        return FAISS.load_local(FAISS_PATH, embed,
                                allow_dangerous_deserialization=True)
    else:
        print("🔨 Creating FAISS index from webpages...")
        web_loader = WebBaseLoader(web_paths=page_urls)
        web_docs = web_loader.load()

        text_loader = TextLoader("hermex.txt", encoding="utf-8")
        text_docs = text_loader.load()

        all_docs = web_docs + text_docs
        splitter = RecursiveCharacterTextSplitter(chunk_size=1000,
                                                  chunk_overlap=200, 
                                                  multithreaded=True)
        chunks = splitter.split_documents(all_docs)
        vectorstore = FAISS.from_documents(chunks, embed)
        vectorstore.save_local(FAISS_PATH)
    return vectorstore


class HermexAssistant:
    def __init__(self):
        self.llm = llm
        self.embedding_model = embed
        self.user_sessions: dict[str, ChatMessageHistory] = {}
        self.vectorstore = setup_vectorstore()

        self.instructions =  """
            You are Hera, a professional and efficient travel assistant for Hermex Travels.
            Your purpose is to provide accurate, concise, and helpful answers strictly based on the provided context.
            Avoid speculation and only provide facts relevant to Hermex services and the travel/tourism industry.

            Use the following context to answer the user's question. If the answer is not in the context, reply with:
            "I'm sorry, I can not provide such information at the moment."
        """
        
        prompt = ChatPromptTemplate(
            messages=[
                SystemMessagePromptTemplate.from_template(self.instructions),
                MessagesPlaceholder("chat_history"),
                HumanMessagePromptTemplate.from_template("{input}"),
                MessagesPlaceholder("agent_scratchpad"),
            ]
        )

        @tool(response_format="content_and_artifact")
        def retrieve() -> str:
            """
            Retrieve information related to a query to provide an informative response.
            """
            retrieved = self.vectorstore.as_retriever(search_kwargs={"k": 2})
            return retrieved, self.conversational_memory.chat_memory

        @tool(response_format="content")
        def ip_address():
            """
            Retrieve IP-address information of the user
            """
            from requests import get
            return get('https://api.ipify.org').text

        @tool(response_format="content")
        def date_time():
            """
            Retrieve current day, date, and time in a readable format
            """
            from datetime import datetime
            now = datetime.now()
            return now.strftime("%A, %Y-%m-%d %H:%M:%S")

        self.conversational_memory = ConversationBufferWindowMemory(
            memory_key='chat_history',
            k=5,
            return_messages=True
        )

        self.agent_executor = AgentExecutor(
            agent=create_tool_calling_agent(self.llm, [retrieve, date_time, ip_address], prompt),
            tools=[retrieve, date_time, ip_address],
            memory=self.conversational_memory,
        )

    def chat(self, user_id: str, query: str) -> str:
        if user_id not in self.user_sessions:
            self.user_sessions[user_id] = ChatMessageHistory()

        self.conversational_memory.chat_memory = self.user_sessions[user_id]

        self.user_sessions[user_id].add_user_message(query)
        response = self.agent_executor.invoke({'input': query})
        self.user_sessions[user_id].add_ai_message(response['output'])
        return response['output']

    def load_history(self, user_id: str) -> list:
        return self.user_sessions.get(user_id, ChatMessageHistory())

    def clear_chat_history(self, user_id: str):
        if user_id in self.user_sessions:
            del self.user_sessions[user_id]
            return True
        return False


OSError: google/flan-t5-xxl is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`